In [ ]:
import os
import tarfile
from six.moves import urllib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import hashlib
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.preprocessing import OneHotEncoder, CategoricalEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline,FeatureUnion
from pandas.plotting import scatter_matrix
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm
from sklearn.externals import joblib

In [ ]:
# this function downloads the TGZ file from the given URL and extracts the contents(csv file)
def download_file(url,dirname):
    if not os.path.isdir(dirname):
        os.makedirs(dirname)
    outpath = os.path.join(dirname,"housing.tgz")
    urllib.request.urlretrieve(url,outpath)
    file_tgz = tarfile.open(outpath)
    file_tgz.extractall(path=dirname)
    file_tgz.close()

In [ ]:
# this function splits the data into random training and testing splits
def split_train_test(data, test_ratio):
    #generate random permutation of indices
    random_indices = np.random.permutation(len(data))
    test_size = int(len(data) * test_ratio)
    test_indices = random_indices[:test_size]
    train_indices = random_indices[test_size:]
    return data.iloc[train_indices],data.iloc[test_indices]

In [ ]:
#this function checks if a given row(identified by the ID) should be part of training or test set based on last byte of the hash value
def test_check(id_, test_ratio):
    # if lasy byte of hash <256*test_ratio( eg: 51 if test_ratio = 20%)
    return hashlib.md5(np.int64(id_)).digest()[-1] < 256*test_ratio

In [ ]:
#this function splits the data into training and testing splits by generating hashes for each data point
#each data point is then assigned to training or testing test based on the last byte of the hash
def split_train_test_id(data, test_ratio, id_col):
    ids = data[id_col]
    in_test_set = ids.apply(lambda id_: test_check(id_,test_ratio))
    return data.loc[~in_test_set], data.loc[in_test_set]

In [ ]:
file = "~/Desktop/Hands_on_ML/handson-ml/datasets/housing/housing.csv"
housing_df = pd.read_csv(file)
housing_df.info() #gives a description of data

In [ ]:
housing_df['ocean_proximity'].value_counts() #counts of each level for 'ocean_proximity' categorical variable

In [ ]:
housing_df.describe() #summary of numerical attributes, ignoring NULL values

In [ ]:
# to show the plots inline
%matplotlib inline
#histograms of all numerical attributes with 50 bins each
housing_df.hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
#splitting the data into 80% - training, 20% - testing randomly
train_set,test_set = split_train_test(housing_df,0.2)
print(len(train_set),len(test_set))

In [ ]:
#adding an index column to the data frame
housing_df = housing_df.reset_index()

In [ ]:
#splitting the data into 80% - training, 20% - testing by assigning each sample to either training or testing set
train_set,test_set = split_train_test_id(housing_df,0.2,"index")
print(len(train_set),len(test_set))

In [ ]:
#splitting the data into 80% - training, 20% - testing using pre-defined sklearn function
#random_state sets a random generator seed. Same indices will be used for training and testing sets for any data of the same size
train_set, test_set = train_test_split(housing_df, test_size=0.2, random_state = 42)
print(len(train_set),len(test_set))

In [ ]:
################# Splitting the data into 80%-training, 20%-testing by stratifying over different income levels ###################

In [ ]:
#deleting the index column created earlier
housing_df.drop('index',axis=1,inplace=True)

In [ ]:
#converting median income(predictor variable) into categorical
housing_df["income_cat"] = np.ceil(housing_df["median_income"] / 1.5) #dividing by 1.5 to limit number of categories

In [ ]:
#replacing any level >5 with 5
#pandas.where replaces values where given condition is FALSE with given value
housing_df["income_cat"].where(housing_df['income_cat'] < 5,5.0,inplace=True)

In [ ]:
#StratifiedShuffleSplit = provides train/test indices to split data
#n_splits = 1 indicates just one set of training and test indices
split = StratifiedShuffleSplit(n_splits = 1, test_size = 0.2, random_state = 42)
for train_index,test_index in split.split(housing_df,housing_df['income_cat']):
    strat_train_set = housing_df.iloc[train_index]
    strat_test_set = housing_df.iloc[test_index]

In [ ]:
housing_df['income_cat'].value_counts()/len(housing_df)

In [ ]:
strat_train_set['income_cat'].value_counts()/ len(strat_train_set)

In [ ]:
strat_test_set['income_cat'].value_counts()/ len(strat_test_set)

In [ ]:
#dropping the categorical variable
strat_train_set.drop("income_cat",axis=1,inplace=True)
strat_test_set.drop("income_cat",axis=1,inplace=True)

In [ ]:
housing_train_df = strat_train_set.copy()

In [ ]:
fig,ax = plt.subplots() #providing axes explicitly as a workaround for axis clipping issue
#creating a scatter plot of longitude and latitude to visualize housing districts
#alpha=0(transparent), alpha=1(opaque), s=marker_size as population size, c = color as median house value column
housing_train_df.plot(kind="scatter",x="longitude",y="latitude", alpha=0.4,
                     s=housing_train_df['population']/100, label="population",figsize=(10,8),
                     c="median_house_value",cmap=plt.get_cmap("jet"),colorbar=True, ax=ax)
plt.legend()

In [ ]:
#computing pairwise-correlations (Pearson's) among all attributes
corr_mat = housing_train_df.corr()
corr_mat['median_house_value'].sort_values(ascending = False)

In [ ]:
#plotting pairwise scatter plots between selected numerical attributes
attributes = ["median_house_value","median_income","total_rooms","housing_median_age"]
scatter_matrix(housing_train_df[attributes],figsize=(12,8))

In [ ]:
#scatter plot between median_house_value and median_income
housing_train_df.plot(kind="scatter",x="median_income", y="median_house_value",alpha=0.1)

In [ ]:
housing_train_df.columns

In [ ]:
#creating hybrid attributes based on existing ones
housing_train_df['rooms_per_households'] = housing_train_df['total_rooms']/housing_train_df['households'] #no. of rooms per household
housing_train_df['prop_bedrooms'] = housing_train_df['total_bedrooms']/housing_train_df['total_rooms'] #proportion of bedrooms all rooms
housing_train_df['pop_per_household'] = housing_train_df['population']/housing_train_df['households'] #population per household

In [ ]:
#computing pairwise-correlations (Pearson's) among all attributes
corr_mat = housing_train_df.corr()
corr_mat['median_house_value'].sort_values(ascending = False)

In [ ]:
#separating target labels and predictors
housing_train_df = strat_train_set.drop("median_house_value",axis=1) #drop creates a copy and does not affect the original df
housing_labels = strat_train_set["median_house_value"].copy()

In [ ]:
#imputing missing values of numeric attributes using the median
imputer = SimpleImputer(strategy="median")
housing_train_num = housing_train_df.drop("ocean_proximity",axis=1) #removing text attribute column
imputer.fit(housing_train_num) #fit imputer instance to training data.Median values are calculated and stored in statistics_ instance variable
X = imputer.transform(housing_train_num)
housing_train_tr = pd.DataFrame(X, columns = housing_train_num.columns)

In [ ]:
#converting categorical variable into integed using factorize() method
housing_train_cat = housing_train_df['ocean_proximity']
housing_train_cat, housing_categories = housing_train_cat.factorize()

In [ ]:
#one-hot encoding of categorical variable
encoder = OneHotEncoder(categories = "auto")
#fit_transform() expects a 2D array. hence the 1D attribute array needs to be reshaped
#reshape() allows one dimension to be -1, which means "unspecified": value is inferred from length of the array
housing_train_cat_1hot = encoder.fit_transform(housing_train_cat.reshape(-1,1))

In [ ]:
housing_train_cat_1hot

In [ ]:
#converting sparse matrix to a Numpy dense array
housing_train_cat_1hot.toarray()

In [ ]:
housing_train_df = strat_train_set.drop("median_house_value",axis=1) #drop creates a copy and does not affect the original df
housing_train_df.columns

In [ ]:
######################## adding hybrid attributes user custom transformer classes ##############################

In [ ]:
#indexes of columns
rooms_ix, bedrooms_ix, population_ix, household_ix = 4, 5, 6, 7

In [ ]:
#BaseEstimator and TransformerMixin are base classes
class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True): #no *args or **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        #creating the hybrid attributes
        rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
        population_per_household = X[:, population_ix] / X[:, household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            #concatenating the hybrid attributes to the existing ones
            return np.c_[X, rooms_per_household, population_per_household]

In [ ]:
#instantiating CombinedAttributesAdder class
attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
#calling the transform method to add the new columns
housing_extra_attribs = attr_adder.transform(housing_train_df.values)

In [ ]:
################################ using pipelines to sequence the transformations ###################################

In [ ]:
#Class to filter out(or pick) only the given attribute names
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self,X,y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

In [ ]:
num_attribs = list(housing_train_num)
cat_attribs = ['ocean_proximity']

In [ ]:
#creating an sklearn pipeline with sequence of transformations for numerical attribues. 
#It takes a list of name/estimator pairs defining a sequence of steps
#All but the last estimator must be transformers i.e they must have a fit_transform() method
#pipelines fit() method when called, calls fit_transform() sequentially on all transformers, passing output of each call to the next call.
#Until the final estimator is reached for which it just calls fit()
num_pipeline = Pipeline([
    ('selector',DataFrameSelector(num_attribs)),
    ('imputer',SimpleImputer(strategy="median")),
    ('attribs_adder',CombinedAttributesAdder()),
    ('std_scaler',StandardScaler())
])

In [ ]:
#creating an sklearn pipeline with sequence of transformations for categorical attribues. 
#It takes a list of name/estimator pairs defining a sequence of steps
cat_pipeline = Pipeline([
    ('selector',DataFrameSelector(cat_attribs)),
    ('cat_encoder',OneHotEncoder(categories = "auto",sparse=False))
])

In [ ]:
#combining the two pipelines(for numerical and categorical attributes) using FeatureUnion class
#It runs each transformer's (or entire transformer pipelines) transform() method in parallel, waits for their output and concatenates them
full_pipeline = FeatureUnion(transformer_list = [
    ("num_pipeline",num_pipeline),
    ("cat_pipeline",cat_pipeline),
])

In [ ]:
#running the whole pipelines
#housing_train_df.drop('index',axis=1,inplace=True)
housing_final = full_pipeline.fit_transform(housing_train_df)

In [ ]:
housing_final.shape

In [ ]:
####################################### Training a Linear Regression model #############################################

In [ ]:
#fitting a linear regression model
lin_reg = LinearRegression()
lin_reg.fit(housing_final, housing_labels)

In [ ]:
#computing root mean squared error for the linear regression predictions on training data
housing_predictions = lin_reg.predict(housing_final)
lin_mse = mean_squared_error(housing_labels,housing_predictions)
lin_rmse = np.sqrt(lin_mse)
print("Room mean squared error in Dollars",lin_rmse) #case of model underfitting

In [ ]:
#performing k-fold(k=10) Cross-Validation
#cross_val_score expects a utility function(greater is better) than a cost function(lower is better) hence negative sign is used
cv_scores = cross_val_score(lin_reg, housing_final, housing_labels, scoring="neg_mean_squared_error",cv=10)
lin_rmse_scores = np.sqrt(-cv_scores)
print(lin_rmse_scores.mean(),lin_rmse_scores.std())

In [ ]:
####################################### Training a Decision tree regressor model #############################################

In [ ]:
#fitting a decision tree regressor
tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_final,housing_labels)

In [ ]:
#computing root mean squared error for the Decision tree regressor predictions
housing_predictions = tree_reg.predict(housing_final)
tree_mse = mean_squared_error(housing_labels,housing_predictions)
tree_rmse = np.sqrt(tree_mse)
print(tree_rmse) #case of model overfitting

In [ ]:
#performing k-fold(k=10) Cross-Validation
#cross_val_score expects a utility function(greater is better) than a cost function(lower is better) hence negative sign is used
cv_scores = cross_val_score(tree_reg, housing_final, housing_labels, scoring="neg_mean_squared_error",cv=10)
tree_rmse_scores = np.sqrt(-cv_scores)
print(tree_rmse_scores.mean(),tree_rmse_scores.std())

In [ ]:
####################################### Training a Random forest regressor model #############################################

In [ ]:
#fitting a random forest regressor
forest_reg = RandomForestRegressor()
forest_reg.fit(housing_final,housing_labels)

In [ ]:
#computing root mean squared error for the Decision tree regressor predictions
housing_predictions = forest_reg.predict(housing_final)
forest_mse = mean_squared_error(housing_labels,housing_predictions)
forest_rmse = np.sqrt(forest_mse)
print(forest_rmse) #case of model overfitting

In [ ]:
#performing k-fold(k=10) Cross-Validation
#cross_val_score expects a utility function(greater is better) than a cost function(lower is better) hence negative sign is used
cv_scores = cross_val_score(forest_reg, housing_final, housing_labels, scoring="neg_mean_squared_error",cv=10)
forest_rmse_scores = np.sqrt(-cv_scores)
print(forest_rmse_scores.mean(),forest_rmse_scores.std())

In [ ]:
####################################### Training a SV Regression model #############################################

In [ ]:
#fitting a SV regressor - RBF kernel
sv_reg = svm.SVR(gamma='scale')
sv_reg.fit(housing_final,housing_labels)

In [ ]:
#computing root mean squared error for the Decision tree regressor predictions
housing_predictions = sv_reg.predict(housing_final)
sv_mse = mean_squared_error(housing_labels,housing_predictions)
sv_rmse = np.sqrt(sv_mse)
print(sv_rmse) #case of model overfitting

In [ ]:
#performing k-fold(k=10) Cross-Validation
#cross_val_score expects a utility function(greater is better) than a cost function(lower is better) hence negative sign is used
cv_scores = cross_val_score(sv_reg, housing_final, housing_labels, scoring="neg_mean_squared_error",cv=10)
sv_rmse_scores = np.sqrt(-cv_scores)
print(sv_rmse_scores.mean(),sv_rmse_scores.std())

In [ ]:
#fitting a SV regressor - Linear kernel
sv_reg = svm.SVR(gamma='scale',kernel='linear')
sv_reg.fit(housing_final,housing_labels)

In [ ]:
#computing root mean squared error for the Decision tree regressor predictions
housing_predictions = sv_reg.predict(housing_final)
sv_mse = mean_squared_error(housing_labels,housing_predictions)
sv_rmse = np.sqrt(sv_mse)
print(sv_rmse) #case of model overfitting

In [ ]:
#performing k-fold(k=10) Cross-Validation
#cross_val_score expects a utility function(greater is better) than a cost function(lower is better) hence negative sign is used
cv_scores = cross_val_score(sv_reg, housing_final, housing_labels, scoring="neg_mean_squared_error",cv=10)
sv_rmse_scores = np.sqrt(-cv_scores)
print(sv_rmse_scores.mean(),sv_rmse_scores.std())

In [ ]:
#saving models(example)
joblib.dump(sv_reg,'mymodel.pkl')

In [ ]:
###################### Performing a grid search over hyperparameters of Random forest regressor ###########################
# Grid search  performs an exhaustive search on all possible combinations of paramaters (based on cartesian product)

In [ ]:
#this function retrieves the evaulations scores of the specific search type
def show_search_scores(results):
    #viewing all evaluation scores, not just the best ones
    cvres = results.cv_results_
    for mean_score, std, params in zip(cvres["mean_test_score"], cvres["std_test_score"], cvres["params"]):
        print(np.sqrt(-mean_score),np.sqrt(std),params)

In [ ]:
#this function retrieves the feature importance scores from the Random Forest regressor based on the best estimator from the search
def show_feature_scores(results):
    #retrieving the feature importances from the best estimator
    feature_importances = results.best_estimator_.feature_importances_
    extra_attribs = ["rooms_per_hhold","pop_pep_hhold","bedrooms_per_room"]
    cat_encoder = cat_pipeline.named_steps["cat_encoder"]
    cat_one_hot_attribs = list(cat_encoder.categories_[0])
    attributes = num_attribs + extra_attribs + cat_one_hot_attribs
    print(sorted(zip(feature_importances, attributes), reverse=True))

In [ ]:
#setting up two different parameter grids
param_grid = [
    {'n_estimators':[3, 10, 30], 'max_features': [2 ,4 ,6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]}
]

In [ ]:
#initiating a RandomForestRegressor
forest_reg = RandomForestRegressor()

In [ ]:
#initiating the GridSearch on the initiated RandomForestRegressor
grid_search = GridSearchCV(forest_reg, param_grid, cv = 5, scoring='neg_mean_squared_error')
grid_search.fit(housing_final, housing_labels)

In [ ]:
#retrieving the best combination of parameters
grid_search.best_params_

In [ ]:
#retrieving the best estimator
grid_search.best_estimator_

In [ ]:
#viewing all evaluation scores, not just the best ones
show_search_scores(grid_search)

In [ ]:
#retrieving the feature importances from the best estimator
show_feature_scores(grid_search)

In [ ]:
###################### Performing a randomized search over hyperparameters of Random forest regressor ###########################
# Randomized search  picks a random combination of parameters. Number of random searchers depends on n_iter parameter

In [ ]:
#setting up two different parameter grids.
#RandomizedSearch does not accept a list of parameter grids
param_grid = {'n_estimators':[3, 10, 30], 'max_features': [2 ,4 ,6, 8]}

In [ ]:
#initiating the randomozed on the initiated RandomForestRegressor
randomized_search = RandomizedSearchCV(forest_reg, param_grid, n_iter = 10, cv = 5, scoring='neg_mean_squared_error')
randomized_search.fit(housing_final, housing_labels)

In [ ]:
#retrieving the best combination of parameters
randomized_search.best_params_

In [ ]:
#viewing all evaluation scores, not just the best ones
show_search_scores(randomized_search)

In [ ]:
#retrieving the feature importances from the best estimator
show_feature_scores(randomized_search)

In [ ]:
################## Evaluating the model on the test set using the best model and hyperparameters from Grid search #####################

In [ ]:
#selecting the best model based on grid search
final_model = grid_search.best_estimator_

In [ ]:
#preparing the test set by separating labels from attributes
X_test = strat_test_set.drop("median_house_value",axis = 1)
y_test = strat_test_set["median_house_value"].copy()

In [ ]:
#running the transformation pipelines on the test set
X_test_final = full_pipeline.transform(X_test)

In [ ]:
#getting the predictions on test set
y_pred = final_model.predict(X_test_final)

In [ ]:
#getting rmse on the test set
final_mse = mean_squared_error(y_test, y_pred)
final_rmse = np.sqrt(final_mse)
print(final_rmse)